# Data Deduplication using Clustering
**Objective**: Learn and implement data deduplication techniques.

**Task**: Hierarchical Clustering for Deduplication

**Steps**:
1. Data Set: Obtain a dataset containing duplicate employee information.
2. Perform Clustering: Use hierarchical agglomerative clustering to cluster the employee
records.
3. Evaluate Duplicates: Determine duplicates by analyzing the clusters formed.
4. Clean Data: Remove duplicate employee records found during clustering.

In [2]:
# write your code from here
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np

# Step 1: Data Set - Create a sample dataset with duplicate employee information
# We'll introduce some variations to simulate real-world duplicates (e.g., typos, different formats)
data = {
    'EmployeeID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111],
    'Name': [
        'John Doe', 'Jane Smith', 'Peter Jones', 'Alice Brown', 'Robert White',
        'Jon Doe', 'Jane Smyth', 'Peter Jonez', 'Alice Browne', 'Rob White',
        'Johnathan Doe' # A slightly different name, not a direct duplicate
    ],
    'Email': [
        'john.doe@example.com', 'jane.smith@example.com', 'peter.jones@example.com',
        'alice.brown@example.com', 'robert.white@example.com',
        'j.doe@example.com', 'jane.smyth@example.com', 'p.jones@example.com',
        'alice.b@example.com', 'rob.white@example.com',
        'johnathan.doe@example.com'
    ],
    'Department': [
        'HR', 'Sales', 'IT', 'Marketing', 'Finance',
        'Human Resources', 'Sales', 'IT', 'Marketing', 'Finance',
        'HR'
    ],
    'Phone': [
        '111-222-3333', '222-333-4444', '333-444-5555', '444-555-6666',
        '555-666-7777', '111-222-3333', '222-333-4444', '333-444-5555',
        '444-555-6666', '555-666-7777', '111-222-3334'
    ]
}
df = pd.DataFrame(data)
print("Original Dataset:")
print(df)
print("\n" + "="*50 + "\n")

# For deduplication, we'll focus on 'Name' and 'Email' as primary identifiers.
# We'll concatenate them to create a single string representation for each record.
df['Combined_Info'] = df['Name'].str.lower() + " " + df['Email'].str.lower()

# Step 2: Perform Clustering - Use hierarchical agglomerative clustering
# 2.1 Feature Extraction: Convert text data into numerical features using TF-IDF
# TF-IDF (Term Frequency-Inverse Document Frequency) is suitable for text similarity.
vectorizer = TfidfVectorizer().fit_transform(df['Combined_Info'])

# 2.2 Calculate Similarity Matrix: Compute cosine similarity between records
# Cosine similarity measures the cosine of the angle between two non-zero vectors.
# A value closer to 1 indicates higher similarity.
similarity_matrix = cosine_similarity(vectorizer)

# Convert similarity to distance for hierarchical clustering
# Distance = 1 - Similarity. This way, higher similarity means lower distance.
distance_matrix = 1 - similarity_matrix

# Ensure the distance matrix is symmetric and has zeros on the diagonal
# (though cosine_similarity and 1-x will naturally do this for self-comparison)
np.fill_diagonal(distance_matrix, 0)

# Perform hierarchical clustering using 'ward' linkage
# 'ward' minimizes the variance of the clusters being merged.
# It's generally good for creating compact, spherical clusters.
linked = linkage(distance_matrix, method='ward')

# Step 3: Evaluate Duplicates - Determine duplicates by analyzing the clusters formed.
# We need to choose a threshold to cut the dendrogram and form clusters.
# A smaller threshold means more clusters (less aggressive deduplication).
# A larger threshold means fewer clusters (more aggressive deduplication).
# For this example, let's choose a threshold that groups records with high similarity.
# A common approach is to inspect the dendrogram or try different thresholds.
# Let's start with a threshold of 0.5 (meaning records with a distance less than 0.5 are grouped).
# This corresponds to a similarity of 0.5 or more.
threshold = 0.5 # Adjust this value based on desired strictness of deduplication

# fcluster forms flat clusters from the hierarchical clustering defined by 'linked'.
# 'criterion='distance'' means clusters are formed when the distance between them is below 'threshold'.
clusters = fcluster(linked, threshold, criterion='distance')

# Add cluster IDs to the DataFrame
df['ClusterID'] = clusters

print("Dataset with Cluster IDs:")
print(df[['EmployeeID', 'Name', 'Email', 'ClusterID']])
print("\n" + "="*50 + "\n")

# Identify duplicates: Records within the same cluster (and cluster size > 1) are considered duplicates.
duplicate_clusters = df.groupby('ClusterID').filter(lambda x: len(x) > 1)

print("Identified Duplicate Records (within clusters of size > 1):")
print(duplicate_clusters[['EmployeeID', 'Name', 'Email', 'ClusterID']])
print("\n" + "="*50 + "\n")

# Step 4: Clean Data - Remove duplicate employee records found during clustering.
# For each cluster, we'll keep only one record (e.g., the first one encountered or the one with the lowest EmployeeID).
# Here, we'll keep the first record in each cluster.
clean_df = df.groupby('ClusterID').first().reset_index()

# Drop the 'Combined_Info' column as it was just for internal processing
clean_df = clean_df.drop(columns=['Combined_Info'])

print("Cleaned Dataset (Duplicates Removed):")
print(clean_df[['EmployeeID', 'Name', 'Email', 'Department', 'Phone']])
print("\n" + "="*50 + "\n")

# Verification: Check if any of the original duplicates are still present in the cleaned data
original_duplicate_pairs = [
    ('John Doe', 'Jon Doe'),
    ('Jane Smith', 'Jane Smyth'),
    ('Peter Jones', 'Peter Jonez'),
    ('Alice Brown', 'Alice Browne'),
    ('Robert White', 'Rob White')
]

print("Verification of Cleaned Data:")
for name1, name2 in original_duplicate_pairs:
    if name1 in clean_df['Name'].values and name2 in clean_df['Name'].values:
        print(f"Warning: '{name1}' and '{name2}' still present in cleaned data. Threshold might need adjustment.")
    elif name1 in clean_df['Name'].values or name2 in clean_df['Name'].values:
        print(f"'{name1}' or '{name2}' (one of the pair) is present, which is expected.")
    else:
        print(f"Neither '{name1}' nor '{name2}' found, indicating successful deduplication for this pair.")

# To see the actual records that were removed:
removed_records = df[~df['EmployeeID'].isin(clean_df['EmployeeID'])]
print("\nRecords that were removed as duplicates:")
print(removed_records[['EmployeeID', 'Name', 'Email', 'ClusterID']])


Original Dataset:
    EmployeeID           Name                      Email       Department  \
0          101       John Doe       john.doe@example.com               HR   
1          102     Jane Smith     jane.smith@example.com            Sales   
2          103    Peter Jones    peter.jones@example.com               IT   
3          104    Alice Brown    alice.brown@example.com        Marketing   
4          105   Robert White   robert.white@example.com          Finance   
5          106        Jon Doe          j.doe@example.com  Human Resources   
6          107     Jane Smyth     jane.smyth@example.com            Sales   
7          108    Peter Jonez        p.jones@example.com               IT   
8          109   Alice Browne        alice.b@example.com        Marketing   
9          110      Rob White      rob.white@example.com          Finance   
10         111  Johnathan Doe  johnathan.doe@example.com               HR   

           Phone  
0   111-222-3333  
1   222-333-4444  


/tmp/ipykernel_2552/2854009314.py:65: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linked = linkage(distance_matrix, method='ward')
